# Linear Regression Implementation

In this intermeiate level:

Addition from basic level. I've implemented Multiple variables, Feature Scaling, Feature engineering with pure mathematic, no framework

TOC:
* [Setup](#setup)
* [Load and analyze data](#load-analyze)
* [Features Scaling](#feature-scaling)
* [Features Engineering](#feature-engineering)
* [Baseline](#baseline)
* [Linear regression](#linear-regression)
* [Cost function w Regularization](#cost)
* [Gradient Descent w Regularization](#gradient-descent)

## Set up <a id='setup'></a>

In [2]:
import numpy as np
import pandas as pd
from sklearn import model_selection, metrics

# Use plotly as it is an interaction plot
import plotly.express as px
# sub plot
from plotly.subplots import make_subplots
import plotly.graph_objects as go

## Load and analyze data <a id='load-analyze'></a>

Load data

In [3]:
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y = raw_df.values[1::2, 2]

In [4]:
# To make sure you understand the data. Read dataspec first.

# Variables in order:

#  X_dataset
#  CRIM     per capita crime rate by town
#  ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
#  INDUS    proportion of non-retail business acres per town
#  CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
#  NOX      nitric oxides concentration (parts per 10 million)
#  RM       average number of rooms per dwelling
#  AGE      proportion of owner-occupied units built prior to 1940
#  DIS      weighted distances to five Boston employment centres
#  RAD      index of accessibility to radial highways
#  TAX      full-value property-tax rate per $10,000
#  PTRATIO  pupil-teacher ratio by town
#  B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
#  LSTAT    % lower status of the population

#  y_dataset
#  MEDV     Median value of owner-occupied homes in $1000's

Train, Test split

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [6]:
# We got X_train np.array with row:404, col:13
print(f"X Shape: {X_train.shape}, X Type:{type(X_train)})")
# We got y_train np.array with row:404
print(f"y Shape: {y_train.shape}, y Type:{type(y_train)})")

X Shape: (303, 13), X Type:<class 'numpy.ndarray'>)
y Shape: (303,), y Type:<class 'numpy.ndarray'>)


Create pd.Dataframe from np.array for readability

In [7]:
X_train_df = pd.DataFrame(X_train, columns=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'])
y_train_df = pd.DataFrame(y_train, columns=['MEDV'])
y_test_df = pd.DataFrame(y_test, columns=['MEDV'])

In [51]:
X_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     303 non-null    float64
 1   ZN       303 non-null    float64
 2   INDUS    303 non-null    float64
 3   CHAS     303 non-null    float64
 4   NOX      303 non-null    float64
 5   RM       303 non-null    float64
 6   AGE      303 non-null    float64
 7   DIS      303 non-null    float64
 8   RAD      303 non-null    float64
 9   TAX      303 non-null    float64
 10  PTRATIO  303 non-null    float64
 11  B        303 non-null    float64
 12  LSTAT    303 non-null    float64
dtypes: float64(13)
memory usage: 30.9 KB


Take a look at data

In [10]:
X_train_df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.03768,80.0,1.52,0.0,0.404,7.274,38.3,7.3090,2.0,329.0,12.6,392.20,6.62
1,2.37934,0.0,19.58,0.0,0.871,6.130,100.0,1.4191,5.0,403.0,14.7,172.91,27.80
2,1.34284,0.0,19.58,0.0,0.605,6.066,100.0,1.7573,5.0,403.0,14.7,353.89,6.43
3,1.65660,0.0,19.58,0.0,0.871,6.122,97.3,1.6180,5.0,403.0,14.7,372.80,14.10
4,0.14030,22.0,5.86,0.0,0.431,6.487,13.0,7.3967,7.0,330.0,19.1,396.28,5.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,0.28392,0.0,7.38,0.0,0.493,5.708,74.3,4.7211,5.0,287.0,19.6,391.13,11.74
299,0.08664,45.0,3.44,0.0,0.437,7.178,26.3,6.4798,5.0,398.0,15.2,390.49,2.87
300,0.15098,0.0,10.01,0.0,0.547,6.021,82.6,2.7474,6.0,432.0,17.8,394.51,10.30
301,0.22927,0.0,6.91,0.0,0.448,6.030,85.5,5.6894,3.0,233.0,17.9,392.74,18.80


In [11]:
y_train_df

,MEDV
0,34.6
1,13.8
2,24.3
3,21.5
4,24.4
...,...
298,18.5
299,36.4
300,19.2
301,16.6


Plot

In [12]:
# Function: Plot and see relationship
def plot_relation(X: pd.DataFrame, y: pd.DataFrame, columns):
    '''
    Plot relation between X input and y target
    
    Args:
        X (pd.DataFrame (m,n))  : Data, m examples, n features
        y (pd.DataFrame (m,1))  : target values, m values
        columns (int)             : number of desired subplot column
        
    Output
        Interation graph
    
    '''
    
    m = X.shape[1]
    rows = m // columns     # Get row
    frac = m % columns      # Get fractual
    row = 0
    col = 1

    if frac > 0:
        rows += 1
            
    fig = make_subplots(rows=rows, cols=columns)

    for i in range(m):
            
        if row >= rows:
            row = 1
            col += 1
        else:
            row += 1
        
        fig.add_trace(go.Scatter(
            x=X.iloc[:,i],
            y=y[y.columns[0]],
            mode='markers',
            name=X.columns[i],
            customdata=X.index.values,                                  # Add customdata for data's row index for more convinient to analysis
            hovertemplate="index:%{customdata} (X: %{x}, y: %{y})"
        ), row=row, col=col)

    fig.update_layout(height=400 * rows, width=600 * columns, title_text='Relationship between All features / ' + y.columns.values[0])
    fig.show()

In [13]:
plot_relation(X_train_df, y_train_df, 2)

## Features scaling <a id='feature-scaling'></a>

Features scaling rescale data with too large/small values to be in range that close to each feature.
And prevent large values influence to paremeters and help model converge more faster

From the dataset minimum value range form 0-1 maximum from 0-700. So scaling would help.

### Z-score normalization

$ x^{(i)}_j = \frac{x^{(i)}_j - \mu_j }{ \sigma_j}$

$ \mu_j = \frac{1}{m} \sum_{i=0}^{m-1} x^{(i)}_j $

$ \sigma^2 = \frac{1}{m} \sum_{i=0}^{m-1} (x^{(i)}_j - \mu_j)^2 $

In [14]:
# Z-score normalization loop
def zscore_normalize_features(X):
    
    m = X.shape[0]
    n = X.shape[1]
    
    mu = np.zeros(n)
    sigma = np.zeros(n)
    X_norm = np.zeros((m,n))
    

    
    for j in range(n):
        
        x_j_sum = 0
        sigma_j_sum = 0
        
        for i in range(m):
            x_j_sum += X[i][j]
        mu[j] = x_j_sum / m
        
        for i in range(m):
            sigma_j_sum += (X[i][j] - mu[j]) ** 2
        sigma[j] = (sigma_j_sum / m) ** (1/2)

        for i in range(m):
            X_norm[i][j] = (X[i][j] - mu[j]) / sigma[j]
            
    return (X_norm, mu, sigma)

In [15]:
# Z-score normalization np
def zscore_normalize_features(X: np.array):
    '''
    Feature scaling: Z-score normalize
    Args:
        X       (np.array (m,n))    : Data, m,n examples
    Returns
        X_norm  (np.array (m,n))    : Data with z-score normallized, m,n examples
    '''
    
    # Mean
    mu = np.mean(X, axis=0)
    # Standard deviation
    sigma = np.std(X, axis=0)
    # Z-score normalize
    X_norm = (X - mu) / sigma   
            
    return X_norm, mu, sigma

Compute and check if min and max is changed

In [16]:
X_train_zscore, X_train_mu, X_train_sigma = zscore_normalize_features(X_train)
armin, armax = np.min(X_train_zscore[0]), np.max(X_train_zscore[0])
print(f'min: {armin}, max: {armax}')

min: -2.6578165547515527, max: 2.7830021145579846


Why we have to return X_train_mu, X_train_sigma?

Because the test set have to scale with this values. Bacause we have to handled test set blindly.

In [17]:
X_test_zscore = (X_test - X_train_mu) / X_train_sigma
X_test_zscore 

array([[-0.38785998, -0.4948566 , -1.12002959, ..., -0.70789303,
         0.16462128, -0.72492749],
       [ 0.6770233 , -0.4948566 ,  0.99479121, ...,  0.78855991,
         0.05643024, -0.41719573],
       [-0.38239792, -0.4948566 ,  0.39563942, ..., -0.93462832,
         0.38589941, -0.28089445],
       ...,
       [-0.39078912,  0.65239395,  0.55930322, ..., -0.11838127,
         0.40106527, -0.60970372],
       [-0.30240613, -0.4948566 , -0.42267953, ...,  1.15133638,
        -0.86498481, -0.11367947],
       [-0.33361345,  0.32460808, -1.01613867, ..., -2.47642832,
         0.32177294, -0.73616883]])

In [18]:
X_train_zscore_df  = pd.DataFrame(X_train_zscore, columns=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'])
X_test_zscore_df  = pd.DataFrame(X_test_zscore, columns=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'])

In [19]:
X_train_zscore_df.agg(['min','max'])

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
min,-0.394938,-0.494857,-1.515669,-0.272888,-1.456259,-3.848304,-2.334317,-1.235741,-0.961476,-1.270760,-2.657817,-4.312171,-1.513227
max,9.942423,3.602467,2.366721,3.664502,2.675413,3.472104,1.078473,4.008098,1.691147,1.804913,1.604807,0.423635,3.439990


You will see that X scale is between -4 to 10, much better than 0 to 700

Notice our categorical data CHAS with binary 0,1 change to -0.2 - 3.89 it is good or bad?

## Feature engineering <a id='feature-engineering'></a>

We do Feature engineering after scaling because if you have large $ X $ ie. 400
in 6 degree polynomial will become $400^6$ but after scaling 400 will become $1.2$  and $1.2^6$ is much smaller than $400^6$.

We've seen in previos plot (original dataset) that some of charts are not in straight line.
The model will perform better if we do the Feature engineering by apply equation that match the shape of charts.

For example $ -X^{\frac{1}{2}} $ in CRIM, $ X^2 $ in DIS after we do Feature Scaling, the chart will become more straight line like equaiton $ y = w * x $

This method require you to read data manually and would cause underfitting

Another method is to multiple every features(x) by degree of polynomial
for example $ [1, 2, 3] $ 3 degree => $ [1, 1^2, 1^3, 2^1, 2^2, 2^3, 3^1, 3^2, 3^3] $ => $ [1, 1, 1, 2, 4, 8, 3, 9, 27] $
and let's the Grdient Descent adjust the weight by in/decreasing $ w $

In [20]:
# Method 1
# X_train_feature = np.c_[-X_train[:,0] ** (1/2), X_train[:,1] ** 2, -X_train[:,2] ** (1/2), X_train[:,3], -X_train[:,4] ** (1/2), X_train[:,5], -X_train[:,6], X_train[:,7] ** 2, X_train[:,8], X_train[:,9], X_train[:,10], X_train[:,11] ** 2, -X_train[:,12] ** (1/2)]
# X_test_feature = np.c_[-X_test[:,0] ** (1/2), X_test[:,1] ** 2, -X_test[:,2] ** (1/2), X_test[:,3], -X_test[:,4] ** (1/2), X_test[:,5], -X_test[:,6], X_test[:,7] ** 2, X_test[:,8], X_test[:,9], X_test[:,10], X_test[:,11] ** 2, -X_test[:,12] ** (1/2)]

# X_train_feature_df = pd.DataFrame(X_train_feature)
# X_test_feature_df = pd.DataFrame(X_test_feature)

In [21]:
def feature_engineering(X: np.array, degree: int):
    
    m = X.shape[0]
    n = X.shape[1]

    X_out = np.zeros([m,n * degree], dtype=np.float64)
        
    for j in range(n):              # 13
        for i in range(degree):     # say 2
            k = (j * degree) + i
            X_out[:,k] = X[:,j] ** (i+1)
            
    return X_out

In [22]:
#bypass scaling
# X_train_zscore = X_train
# X_test_zscore = X_test

# Degree of polynomial
degree = 2

X_train_z_feature = feature_engineering(X_train_zscore, degree)
X_test_z_feature = feature_engineering(X_test_zscore, degree)

X_train_z_feature_df = pd.DataFrame(X_train_z_feature)
X_test_z_feature_df = pd.DataFrame(X_test_z_feature)

In [23]:
X_test_z_feature_df

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,-0.387860,0.150435,-0.494857,0.244883,-1.120030,1.254466,-0.272888,0.074468,-0.818655,0.670196,...,-0.615482,0.378818,0.155554,0.024197,-0.707893,0.501113,0.164621,0.027100,-0.724927,0.525520
1,0.677023,0.458361,-0.494857,0.244883,0.994791,0.989610,-0.272888,0.074468,0.635081,0.403328,...,1.691147,2.859978,1.540781,2.374006,0.788560,0.621827,0.056430,0.003184,-0.417196,0.174052
2,-0.382398,0.146228,-0.494857,0.244883,0.395639,0.156531,3.664502,13.428571,-0.053531,0.002866,...,-0.500150,0.250150,-0.748365,0.560050,-0.934628,0.873530,0.385899,0.148918,-0.280894,0.078902
3,2.486023,6.180312,-0.494857,0.244883,0.994791,0.989610,-0.272888,0.074468,1.162167,1.350632,...,1.691147,2.859978,1.540781,2.374006,0.788560,0.621827,0.423635,0.179467,1.021696,1.043863
4,-0.389113,0.151409,-0.494857,0.244883,0.244784,0.059919,-0.272888,0.074468,-1.014187,1.028575,...,-0.500150,0.250150,-0.032273,0.001042,0.108354,0.011741,0.298248,0.088952,-0.022344,0.000499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,-0.303835,0.092316,-0.494857,0.244883,-0.172203,0.029654,-0.272888,0.074468,-0.104539,0.010928,...,-0.615482,0.378818,-0.584016,0.341075,-0.027687,0.000767,0.423635,0.179467,-0.916030,0.839112
199,-0.388551,0.150972,0.324608,0.105370,-1.107221,1.225939,3.664502,13.428571,-0.964029,0.929351,...,-0.500150,0.250150,-1.100541,1.211191,-1.614834,2.607690,0.186833,0.034906,-1.333365,1.777863
200,-0.390789,0.152716,0.652394,0.425618,0.559303,0.312820,-0.272888,0.074468,-0.784650,0.615675,...,-0.615482,0.378818,-0.783582,0.614002,-0.118381,0.014014,0.401065,0.160853,-0.609704,0.371739
201,-0.302406,0.091449,-0.494857,0.244883,-0.422680,0.178658,-0.272888,0.074468,-0.155547,0.024195,...,-0.615482,0.378818,-0.566407,0.320817,1.151336,1.325575,-0.864985,0.748199,-0.113679,0.012923


In [24]:
# plot_relation(X_train_z_feature_df, y_train_df, 2)

You will see that all graph have the same proportion as the original dataset

Before going to the prediction.
we need  some value to benchmark our model called
## Baseline Model <a id='baseline'></a>

Baseline model is somehow complex and I will explain in saperate topic.

For quick explainaiton. It is some value you use to benchmark the performance of your model.

You might use MSE from simple linear regression as baseline. Or use mean from target as I show below as baseline.
Which it has MSE:84.62 which I think it's not a good baseline. Imagine you predict 84,000$ house's price for actual 1,000$. A company will bankrupt in no time.

So maybe adding simple linear regression is better (For sake of simplisity I will add in advanced topic).

In [25]:
# Mean of y
y_train_mean = y_train.mean()
y_pred = [y_train_mean] * len(y_train)

In [26]:
# Test model Evaluation
print('R^2:',metrics.r2_score(y_train, y_pred))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_train, y_pred))
print('MSE:',metrics.mean_squared_error(y_train, y_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_train, y_pred)))

R^2: 0.0
Adjusted R^2: -0.04498269896193774
MAE: 6.733483645394243
MSE: 85.43727346992125
RMSE: 9.243228519836629


Simple linear regression score. I use sklearn to keep code simple.

In [27]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(X_train, y_train)
# reg.score(X, y)
# reg.coef_
# reg.intercept_

y_train_baseline =  reg.predict(X_train)

In [28]:
print('R^2:',metrics.r2_score(y_train, y_train_baseline))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_train, y_train_baseline))*(len(y_train_baseline)-1)/(len(y_train_baseline)-X_train.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_train, y_train_baseline))
print('MSE:',metrics.mean_squared_error(y_train, y_train_baseline))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_train, y_train_baseline)))

R^2: 0.7668160223286261
Adjusted R^2: 0.7563267776582875
MAE: 3.118896450030036
MSE: 19.922603269113182
RMSE: 4.4634743495525075


In [29]:
y_test_baseline =  reg.predict(X_test)
print('R^2:',metrics.r2_score(y_test, y_test_baseline))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_test, y_test_baseline))*(len(y_test_baseline)-1)/(len(y_test_baseline)-X_test.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_test, y_test_baseline))
print('MSE:',metrics.mean_squared_error(y_test, y_test_baseline))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_test_baseline)))

R^2: 0.6882607142538013
Adjusted R^2: 0.6668183295199358
MAE: 3.6331273740246157
MSE: 25.790362150702496
RMSE: 5.078421226198404


# Linear regression <a id='linear-regression'></a>

### Formula

$ f_{w,b}(x^{(i)}) = wx^{(i)} + b $

In [30]:
# Vectorized implementation
def compute_linear_regression_v(X, w, b):
    f_wb = X.dot(w.T) + b
    return f_wb

In [31]:
w_init = np.ones(X_train_z_feature.shape[1])
b_init = 1
f_wb = compute_linear_regression_v(X=X_train_z_feature, w=w_init , b=b_init)

# Cost function with Regularization <a id='cost'></a>

Problem: if parameter $w$ is too large, it's would havily influence the model, hence would cause overfitting.
To fix this ploblem Regularization is involvded by compute on $ w $ as shown after simple cost functio below

For human explaination it is &Cost(Error)& increase with mean of $w$ squared with some multiply rate.

if $w$ is too large $Cost$ will increase significantly to decrease $w$ later in Gradient Descent to balance all the parameters.

$ J(w, b) = \frac{1}{2m} \sum_{i=1}^{m} (f_{w,b}(x^{(i)}) - y^{(i)})^2 + \frac{\lambda}{2m}\sum^{n-1}_{j=0}w^2_j $

In [32]:
def compute_cost_v(X, y, w, b, lambda_):
    f_wb = compute_linear_regression_v(X, w, b)
    reg = lambda_ / 2 * sum(w ** 2).mean()
    cost = ((f_wb - y) ** 2).mean() / 2 + reg
    return cost

In [33]:
compute_cost_v(X_train_z_feature, y_train, w=w_init, b=b_init, lambda_=0.01)

201.96549362405355

## Gradeint Descent <a id='gradient-descent'></a>

$ \{ $
    
$ w^{(i)}_j := w^{(i)}_j - \alpha \frac{\sigma}{\sigma w} [J(w, b)^2 x^{(i)}_j + \frac{\lambda}{2m}\sum^{n-1}_{j=0}w^2_j] $

$ b^{(i)} := b^{(i)} - \alpha \frac{\sigma}{\sigma w}J(w, b) $

$ \} {stimulous update} $


$ = \alpha \frac{1}{m} \sum_{i=1}^{m} (f_{(w,b)}(x^{(i)}) - y^{(i)}) + \alpha \frac{\lambda}{m}\sum^{n-1}_{j=0}w_j $

In [34]:
def gradient_function_w_reg_v(X, y, w, b, lambda_):
    
    m = X.shape[0]
    
    dj_dw = 0
    dj_db = 0

    f_wb = compute_linear_regression_v(X, w, b)
    error = f_wb - y
    reg = lambda_ * sum(w)
    dj_dw = error.T.dot(X)
    dj_db = sum(error)
    dj_dw = (dj_dw + reg) / m
    dj_db = dj_db / m
        
    return dj_dw, dj_db

In [35]:
dj_dw, dj_db = gradient_function_w_reg_v(X_train_z_feature, y_train, w=w_init, b=b_init, lambda_=0.01)

In [36]:
def gradient_descent(X, y, w, b, alpha, num_iters, cost_function, gradient_function, lambda_):
    
    j_hist = []
    p_hist = []

    for i in range(num_iters):
        
        dj_dw, dj_db = gradient_function(X, y, w, b, lambda_)
        
        w = w - np.dot(alpha, dj_dw)
        b = b - np.dot(alpha, dj_db)

        j = cost_function(X, y, w, b, lambda_)
        
        # stop when converge (j_old - j_new < thresold)
        if (len(j_hist) > 100):
            if ((j_hist[i-1] - j) < 0.01):
                if i % 1000 == 0:
                    j_hist.append(j)
                    p_hist.append([w,b])
                break
        
        if i % 100 == 0:
            j_hist.append(j)
            p_hist.append([w,b])
        
    return w, b, j_hist, p_hist

In [37]:
w_out, b_out, j_hist, p_hist = gradient_descent(X_train_z_feature, y_train, w=w_init, b=b_init,alpha=0.01, num_iters=10000, cost_function=compute_cost_v, gradient_function=gradient_function_w_reg_v, lambda_=0.01)

Let's check is decrease gradually

In [38]:
j_hist

[153.61735899714625,
 20.81110429900291,
 14.866756936273378,
 12.477875081219842,
 11.07133348524422,
 10.147653121466995,
 9.50451929674334,
 9.0393398587695,
 8.692694407726318,
 8.427304688098411,
 8.218745443437102,
 8.05063449272263,
 7.911829590374214,
 7.794674129711822,
 7.693856740761525,
 7.60565538627129,
 7.527431125145092,
 7.4572870227728005,
 7.393837373585161,
 7.336051023626188,
 7.283144652306773,
 7.234509832087514,
 7.189662981596495,
 7.148210871343003,
 7.109826720276917,
 7.0742335228247875,
 7.041192326240015,
 7.010493908142493,
 6.981952798383164,
 6.95540292457214,
 6.930694388320973,
 6.907691034197069,
 6.886268578998261,
 6.866313141060052,
 6.847720058623493,
 6.830392920083516,
 6.814242752139251,
 6.799187327834309,
 6.7851505674910655,
 6.7720620131669085,
 6.759856362556351,
 6.748473051958536,
 6.737855880523182,
 6.7279526698211685,
 6.718714954092912,
 6.710097697469765,
 6.702059035151683,
 6.694560036034597,
 6.687564484666258,
 6.68103868070593

In [39]:
print(f'result cost:{min(j_hist)} with parameter w:{w_out} b:{b_out}')

result cost:6.595774365033364 with parameter w:[-3.18415043  0.21664914 -1.29246014  0.57238179  0.06197643  0.551773
 -3.42949495  1.20333485 -2.9794363   0.13146356  1.60511132  0.65121246
  0.35993064  0.1526612  -3.09685459  0.31762777  3.3298905   0.1936863
 -2.22450171  0.04749202 -1.57782738  0.77736567  0.4161224  -0.01202141
 -5.96955283  1.63223212] b:16.299387166103273


Test with gradient descent output w, b parameters

In [40]:
compute_cost_v(X_train_z_feature, y_train, w=w_out, b=b_out, lambda_=0.01)

6.595674735265872

Now let's check with unknown data (Test dataset)

In [41]:
compute_cost_v(X_test_z_feature, y_test, w=w_out, b=b_out, lambda_=0.01)

10.223437144621

#### Let's plot Cost function

In [42]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Scatter(
        x=[i for i in range(len(j_hist))],
        y=j_hist,
        mode='lines+markers'
        ))

# include shapes in layout
fig.update_layout(height=400, width=600, title_text="Cost Function")
fig.show()

Cost not decrease significantly after 100 iteration

Maybe we should consider to set threshold to interupt iteration after reach certain value ($ \epsilon $)

In [43]:
print(f'result cost:{min(j_hist)} with parameter w:{w_out} b:{b_out}')

result cost:6.595774365033364 with parameter w:[-3.18415043  0.21664914 -1.29246014  0.57238179  0.06197643  0.551773
 -3.42949495  1.20333485 -2.9794363   0.13146356  1.60511132  0.65121246
  0.35993064  0.1526612  -3.09685459  0.31762777  3.3298905   0.1936863
 -2.22450171  0.04749202 -1.57782738  0.77736567  0.4161224  -0.01202141
 -5.96955283  1.63223212] b:16.299387166103273


In [44]:
y_train_pred = np.dot(X_train_z_feature, w_out) + b_out
y_test_pred = np.dot(X_test_z_feature, w_out) + b_out
y_train_pred_df = pd.DataFrame(y_train_pred)
y_test_pred_df = pd.DataFrame(y_test_pred)

Let's check our result with parameter w,b

and plot to see how the model fit the targets

In [45]:
def plot_result(X: pd.DataFrame, y: pd.DataFrame, y_pred: pd.DataFrame, columns):
    '''
    Plot relation between X input and y target
    
    Args:
        X (pd.DataFrame (m,n))  : Data, m examples, n features
        y (pd.DataFrame (m,1))  : target values, m values
        columns (int)             : number of desired subplot column
        
    Output
        Interation graph
    
    '''
    
    m = X.shape[1]
    rows = m // columns     # Get row
    frac = m % columns      # Get fractual
    row = 0
    col = 1

    if frac > 0:
        rows += 1
            
    fig = make_subplots(rows=rows, cols=columns)

    for i in range(m):
            
        if row >= rows:
            row = 1
            col += 1
        else:
            row += 1
        
        fig.add_trace(go.Scatter(
            x=X.iloc[:,i],
            y=y[y.columns[0]],
            mode='markers',
            name=X.columns[i],
            customdata=X.index.values,                                  # Add customdata for data's row index for more convinient to analysis
            hovertemplate="index:%{customdata} (X: %{x}, y: %{y})"
        ), row=row, col=col)
        
        fig.add_trace(go.Scatter(
            x=X.iloc[:,i],
            y=y_pred[y_pred.columns[0]],
            mode='markers',
            name=X.columns[i],
            customdata=X.index.values,                                  # Add customdata for data's row index for more convinient to analysis
            hovertemplate="index:%{customdata} (X: %{x}, y: %{y})"
        ), row=row, col=col)

    fig.update_layout(height=400 * rows, width=600 * columns, title_text='Relationship between All features / ' + y.columns.values[0])
    fig.show()

In [46]:
plot_result(X_train_z_feature_df, y_train_df, y_train_pred_df, 2)

Plot again with test set

In [47]:
plot_result(X_test_z_feature_df, y_test_df, y_test_pred_df, 2)

In [48]:
# Train model Evaluation
print('R^2:',metrics.r2_score(y_train, y_train_pred))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_train, y_train_pred))*(len(y_train)-1)/(len(y_train)-X_test.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_train, y_train_pred))
print('MSE:',metrics.mean_squared_error(y_train, y_train_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_train, y_train_pred)))

R^2: 0.8579014681557521
Adjusted R^2: 0.851509492674869
MAE: 2.44489351949556
MSE: 12.140511124851319
RMSE: 3.4843236251604583


In [49]:
# Test model Evaluation
print('R^2:',metrics.r2_score(y_test, y_test_pred))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_test, y_test_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_test, y_test_pred))
print('MSE:',metrics.mean_squared_error(y_test, y_test_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

R^2: 0.7655517066405838
Adjusted R^2: 0.7494256335523699
MAE: 3.120282065174754
MSE: 19.396035943561575
RMSE: 4.4040930897929


Our prediction performance is better than baseline, goodjob!